 python version：3.8   
 modules：pandas

这个文件主要负责处理流图所需数据

流图展示资产变动，需要的数据为每一时刻每个币种的价值

某时刻某币价值=某币持有数*某时刻单价

现简单考虑，用最低交易所收购价格(bid1_price)作为那一时刻单价

1. 读取数据

In [1]:
import os
import pandas as pd
dir='../cache/'
BTC_data = pd.read_hdf(dir+'BTC_Trade.hdf', 'data')
BNB_data = pd.read_hdf(dir+'BNB_Trade.hdf', 'data')
ETH_data = pd.read_hdf(dir+'ETH_Trade.hdf', 'data')
deal=pd.read_excel(dir+'交易结果表单2.xlsx')

2. 合并

In [2]:
import pandas as pd
import time
def Merge(deal,market_data):
    '''
    deal：交易表单，其中Time字段元素为交易完成时间，start time元素为交易发起时间，filled_size表示实际成交量
    market_data：市场数据，其中bid1_price表示交易所最低收购价格
    '''

    ## 合并相同时刻记录
    deal=deal.groupby(['Time']).agg({'filled_size': 'sum','size': 'sum'})

    ## 计算累积持有量
    deal['holdings']=deal['filled_size'].cumsum()


    ## 重采样填充时间轴上的空缺
    deal.index=pd.to_datetime(deal.index)
    deal=deal.resample('1S').ffill()

    ## 时间转化为时间戳
    deal.reset_index(inplace=True)
    deal['time']=(pd.to_datetime(deal['Time']).astype('int64')//1e9).astype('int64')

    ## 市场数据时间戳调整为10位
    market_data['time']=market_data['time'].astype('str').str.slice(0,10).astype('int64')

    ## 去除市场数据时间戳重复
    market_data=market_data.drop_duplicates(subset=['time'],keep='first')

    ## 跟市场数据的数据框按时间戳进行合并
    merged_df=pd.merge(deal, market_data, on='time', how='left')
    # print(merged_df.isnull().sum()) ## 检查是否有nan
    ## 计算各个时刻持有价值
    merged_df['value'] =  merged_df['holdings'] * merged_df['bid1_price']

    ## 输出关键字段
    return merged_df[['time','holdings','bid1_price','value']]

BTCdf=Merge(deal[deal.currency=='BTC'],BTC_data)
BNBdf=Merge(deal[deal.currency=='BNB'],BNB_data)
ETHdf=Merge(deal[deal.currency=='ETH'],ETH_data)
    

In [3]:
columns=['holdings','bid1_price','value']
for Name in ('BTC','BNB','ETH'):
    globals()[Name+'df'].columns=['time']+[Name+' '+c for c in columns]
result=pd.merge(BTCdf,BNBdf,on='time', how='outer')
result=pd.merge(result,ETHdf,on='time', how='outer')

'''
流图要的数据
'''
result.fillna(method='ffill', inplace=True)
result.fillna(0, inplace=True)
#------------------------ 加入美元价值
### 合并相同时刻记录
USD=deal.groupby(['Time']).agg({'filled_usd_size': 'sum'})
### 计算累积持有量
USD['usd value']=-USD['filled_usd_size'].cumsum()
### 重采样填充时间轴上的空缺
USD.index=pd.to_datetime(USD.index)
USD=USD.resample('1S').ffill()
### 时间转化为时间戳
USD.reset_index(inplace=True)
USD['time']=(pd.to_datetime(USD['Time']).astype('int64')//1e9).astype('int64')
result=pd.merge(result,USD[['time','usd value']],on='time',how='left')
result['bias usd value']=result['usd value']-result['usd value'].min()
#------------------------
for Name in ('BTC','BNB','ETH'):
    result['bias '+Name+' value']=result[Name+' value']-result[Name+' value'].min()
result.to_csv('../frontend/public/dataset/流图数据.csv',index=False)